创建数据库引擎的原因主要是为了使用SQLAlchemy库提供的功能。SQLAlchemy是Python中一个流行的SQL工具包和对象关系映射（ORM）库，它提供了一种高效、高层次的抽象接口来与数据库交云。在您的脚本中，虽然直接使用[pymysql](file:///d%3A/github/myxlnet/d%3A%5Cgithub%5Cmyxlnet%5Cxlnet_lijianye_dataset.ipynb#1%2C8-1%2C8)连接和操作数据库已经足够处理当前的需求，但在某些情况下，使用SQLAlchemy可以带来额外的便利和功能，例如：

In [6]:
from dateutil import parser

date_str = '2024/4/24 0:33:00'
date = parser.parse(date_str)
print(date)

2024-04-24 00:33:00


In [3]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from datetime import datetime
import numpy as np
from dateutil import parser
# 数据库连接信息
host = '38.147.173.234'
user = 'lijianye'
password = '660013'
db = 'training_statistics_db'
table_name = 'training_info'

# 使用SQLAlchemy创建数据库引擎
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db}')

# 读取CSV文件
df = pd.read_csv('modified_train_with_updated_duration.csv')

# 连接到数据库
connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             database=db,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
try:
    with connection.cursor() as cursor:
        for index, row in df.iterrows():
            # 格式化start_time和end_time为正确的DATETIME格式
            start_time_str = row['start_time']  # 分割并取第一部分
            end_time_str = row['end_time']  # 同上
            print(start_time_str)
            # 使用dateutil.parser.parse解析日期时间字符串
            start_time = parser.parse(start_time_str)
            end_time = parser.parse(end_time_str)
            cursor.execute(f"SELECT EXISTS(SELECT 1 FROM {table_name} WHERE start_time = %s)", (start_time_str,))
            result = cursor.fetchone()
            exists = list(result.values())[0]
            
            if not exists:
                duration = row['duration']
                # 准备插入数据，确保所有字段都匹配模型定义
                insert_data = (
                    row['epoch'],
                    start_time.strftime('%Y-%m-%d %H:%M:%S'),
                    end_time.strftime('%Y-%m-%d %H:%M:%S'),
                    duration,
                    int(row['user_id']),
                    row['model'],
                    row['top1_accuracy'],
                    row['top2_accuracy'],
                    row['top3_accuracy'],
                    row['top4_accuracy'],
                    row['top5_accuracy'],
                    row['top6_accuracy'],
                    row['top7_accuracy'],
                    row['top8_accuracy'],
                    row['top9_accuracy'],
                    row['top10_accuracy'],
                    row['optional_feature'],
                    row['learning_rate'],
                    row['dataset'],
                    row['experiment_num']
                )
                
                # 构建SQL插入语句
                sql_insert = f"INSERT INTO {table_name} (epoch, start_time, end_time, duration, user_id, model, top1_accuracy, top2_accuracy, top3_accuracy, top4_accuracy, top5_accuracy, top6_accuracy, top7_accuracy, top8_accuracy, top9_accuracy, top10_accuracy, optional_feature, learning_rate, dataset, experiment_num) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                
                cursor.execute(sql_insert, insert_data)
                connection.commit()
                print(f"Inserted record with start time {start_time} and calculated duration into database.")
            else:
                print(f"Record with start time {start_time} already exists in database, skipping.")
finally:
    connection.close()

2024/5/13 23:08:37
Inserted record with start time 2024-05-13 23:08:37 and calculated duration into database.
2024/5/13 23:28:01
Inserted record with start time 2024-05-13 23:28:01 and calculated duration into database.
2024/5/13 23:51:27
Inserted record with start time 2024-05-13 23:51:27 and calculated duration into database.
2024/5/14 0:15:01
Inserted record with start time 2024-05-14 00:15:01 and calculated duration into database.
2024/5/14 0:39:27
Inserted record with start time 2024-05-14 00:39:27 and calculated duration into database.
2024/5/14 1:02:56
Inserted record with start time 2024-05-14 01:02:56 and calculated duration into database.
2024/5/14 1:26:18
Inserted record with start time 2024-05-14 01:26:18 and calculated duration into database.
2024/5/14 1:50:50
Inserted record with start time 2024-05-14 01:50:50 and calculated duration into database.
2024/5/14 2:14:15
Inserted record with start time 2024-05-14 02:14:15 and calculated duration into database.
2024/5/14 2:35:3

In [4]:
import pymysql

# 数据库连接信息
host = '38.147.173.234'
user = 'lijianye'
password = '660013'
db = 'training_statistics_db'

try:
    # 尝试连接到数据库
    connection = pymysql.connect(host=host,
                                 user=user,
                                 password=password,
                                 database=db,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    print("数据库连接成功！")
    
    # 添加一个新用户到users表
    with connection.cursor() as cursor:
        # 准备SQL语句
        sql = "INSERT INTO users (username, password) VALUES (%s, %s)"
        # 执行SQL语句
        cursor.execute(sql, ('caiye', '123456'))
        # 提交到数据库执行
        connection.commit()
        print("成功添加用户caiye到数据库。")
    
    # 获取数据库版本信息
    with connection.cursor() as cursor:
        cursor.execute("SELECT VERSION()")
        version = cursor.fetchone()
        print("数据库版本：", version)
    
finally:
    # 关闭数据库连接
    connection.close()

数据库连接成功！


IntegrityError: (1062, "Duplicate entry 'caiye' for key 'users.username'")